### Lendo imagem

In [ ]:
import SimpleITK as sitk
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


#root_dir linux
#root_dir ="/home/adriano/projeto_mestrado/modules"

#root_dir windows
root_dir = Path(r"C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari")
dir_images = f'/imagens/3d/'

image_path = f'{root_dir}{dir_images}I7025.nii.gz'
image = sitk.ReadImage(image_path) 

    
# Converter para numpy array
image_array = sitk.GetArrayFromImage(image)

# Exibir um slice no meio do volume
slice_idx = image_array.shape[0] // 2  # Pega o índice do slice central
slice_image = image_array[slice_idx, :, :]

# Plotar o slice
plt.figure(figsize=(8, 8))
plt.imshow(slice_image, cmap="gray")
plt.title(f"Slice {slice_idx}")
plt.axis("off")
plt.show()



### Função

In [2]:
import os
def read_directories(directory, img=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)          
        else:
            filenames.append(filename)    
    return filenames

### Diretório das imagens

In [6]:
dir_images = f'work1/clinical_images'

### Filtro Anisotrópico

In [15]:

from pathlib import Path

import SimpleITK as sitk

def apply_anisotropic_diffusion_sitk(image_path, time_step, conductance, iterations, output_path):
    """
    Apply a 3D anisotropic diffusion filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - time_step (float): The time step for the diffusion process.
    - conductance (float): The conductance parameter for the diffusion process.
    - iterations (int): Number of iterations for the diffusion process.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Cast the image to a supported pixel type (float32)
    image = sitk.Cast(image, sitk.sitkFloat32)
    
    # Apply Anisotropic Diffusion filter
    anisotropic_diffusion_filter = sitk.CurvatureAnisotropicDiffusionImageFilter()
    anisotropic_diffusion_filter.SetTimeStep(time_step)
    anisotropic_diffusion_filter.SetConductanceParameter(conductance)
    anisotropic_diffusion_filter.SetNumberOfIterations(iterations)
    filtered_image = anisotropic_diffusion_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/anisotropicDiffusion'  
    time_step, conductance, iterations =  0.0625, 1.0, 10
    for image in array_images:    
        output_name = f'{image}'
        apply_anisotropic_diffusion_sitk(f'{dir_images}/{image}', time_step, conductance, iterations, f'{dir_output}/{output_name}')
        

Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI002-Guys-0828-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI012-HH-1211-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI013-HH-1212-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI014-HH-1236-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI015-HH-1258-T1.nii.gz


### NLM - Skimage

In [14]:
# Need to install
# pip install scikit-image
# pip install imageio-ffmpeg (not sure if this is needed ???)
# pip install nibabel
# pip install PyWavelets

import numpy as np
from skimage.restoration import denoise_nl_means, estimate_sigma
import nibabel as nib
from pathlib import Path

def apply_nonlocal_means_skimage(image_path, patch_size, patch_distance, h, fast_mode, output_path):
    """
    Apply a 3D Non-Local Means filter to a 3D image using scikit-image.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - patch_size (int): The size of the patches used for denoising.
    - patch_distance (int): The distance for the neighborhood search.
    - h (float): The filtering parameter.
    - fast_mode (bool): Whether to use the fast mode or not.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = nib.load(image_path)
    image_data = image.get_fdata()
    
    # Estimate the noise standard deviation from the noisy image
    sigma_est = np.mean(estimate_sigma(image_data))
    
    # Apply Non-Local Means filter
    denoised_image = denoise_nl_means(image_data, patch_size=patch_size, patch_distance=patch_distance, h=h*sigma_est, fast_mode=fast_mode)
    
    # Save the filtered image
    denoised_image_nib = nib.Nifti1Image(denoised_image, image.affine)
    nib.save(denoised_image_nib, output_path)
    print(f"Filtered image using scikit-image saved to {output_path}")

# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/non_local_means'
    i = 0
    patch_size, patch_distance, h = 3, 5, 1.15
    for image in array_images:    
        output_name = f'{image}'
        apply_nonlocal_means_skimage(f'{dir_images}/{image}',  patch_size, patch_distance, h, True, f'{dir_output}/{output_name}')


Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI002-Guys-0828-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI012-HH-1211-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI013-HH-1212-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI014-HH-1236-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI015-HH-1258-T1.nii.gz


### NLM SimpleITK

In [ ]:
import SimpleITK as sitk
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt



array_images = read_directories(dir_images)
for image in array_images:
    patch_radius = 4
   
    image_path = f'{dir_images}/{image}'
    print(image_path)
    image_read = sitk.ReadImage(image_path)
    # Apply Non-Local Means filter
    nlm_filter = sitk.PatchBasedDenoisingImageFilter()
    nlm_filter.SetPatchRadius(patch_radius)
    nlm_filter.SetNoiseModel(sitk.PatchBasedDenoisingImageFilter.GAUSSIAN)
    filtered_image = nlm_filter.Execute(image_read)
    output_path = f'{dir_images}/denoising/non_local_means/'
    output_name = f'{image}'
    # Save the filtered image
    sitk.WriteImage(filtered_image, f'{output_path}/{output_name}')
  

### Bilateral Filter

In [ ]:
import SimpleITK as sitk

def apply_bilateral_filter_sitk(image_path, domain_sigma, range_sigma, output_path):
    """
    Apply a 3D bilateral filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - domain_sigma (float): The standard deviation for the spatial Gaussian kernel.
    - range_sigma (float): The standard deviation for the range Gaussian kernel.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Bilateral filter
    bilateral_filter = sitk.BilateralImageFilter()
    bilateral_filter.SetDomainSigma(domain_sigma)
    bilateral_filter.SetRangeSigma(range_sigma)
    filtered_image = bilateral_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

if __name__ == "__main__":
    array_images = read_directories(dir_images)
    #root_dir = Path(r"C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari")
    dir_output = f'work1/denoising/bilateral'
    domain_sigma, range_sigma =  2.0, 50.0   

for image in array_images:    
    output_name = f't{image}'
    apply_bilateral_filter_sitk(f'{dir_images}/{image}', domain_sigma, range_sigma, f'{dir_output}/{output_name}')
    


### G - Images

#### bilateral

In [22]:
dir_anisotropic = f'work1/denoising/bilateral/images_filtered'
dir_out_anisotropic = f'work1/denoising/bilateral/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')


#### anisotropicDiffusion

In [23]:
dir_anisotropic = f'work1/denoising/anisotropicDiffusion/images_filtered'
dir_out_anisotropic = f'work1/denoising/anisotropicDiffusion/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')


#### non_local_means

In [25]:
dir_anisotropic = f'work1/denoising/non_local_means/images_filtered'
dir_out_anisotropic = f'work1/denoising/non_local_means/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')
